In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
import torch
from segmentation_pipeline import *

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

1.5277058988770118 4.6075944008792025


In [ ]:
from batchgenerators.utilities.file_and_folder_operations import load_pickle, save_pickle
from pathlib import Path


root = Path("X:/Datasets/nnUNet_raw_data_base/nnUNet_raw_data/Task501_DMRI_Hippocampus_Whole/")

root = Path("X:/Datasets/nnUNet_preprocessed/Task501_DMRI_Hippocampus_Whole/")
dataset_properties = load_pickle(root / "nnUNetPlansv2.1_plans_3D.pkl")
dataset_properties

In [ ]:
import os
import sys

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from segmentation_pipeline import *

In [ ]:
save_dataset_as_nn_unet

In [ ]:
config = load_module("../configs/diffusion_hippocampus.py")
variables = dict(DATASET_PATH="X:/Datasets/Diffusion_MRI/")
context = config.get_context(device, variables)
context.init_components()

In [ ]:
for name in context.dataset.cohorts.keys():
    dataset = context.dataset.get_cohort_dataset(name)
    print(name, len(dataset))

In [ ]:
output_labels = ["whole_roi"]
cbbrain_dataset = context.dataset.get_cohort_dataset(ComposeFilters([
    RequireAttributes(output_labels),
    RequireAttributes({"pathologies": "None", "rescan_id": "None"}),
    RequireAttributes({"protocol": "cbbrain"}),
]))
ab300_unlabeled_dataset = context.dataset.get_cohort_dataset(ComposeFilters([
    ForbidAttributes(output_labels),
    RequireAttributes({"pathologies": "None", "rescan_id": "None"}),
    RequireAttributes({"protocol": "ab300"})
]))

In [ ]:
test_filter = StratifiedFilter(size=53, continuous_attributes=['age'], discrete_attributes=['gender'])
cbbrain_test = cbbrain_dataset.get_cohort_dataset(test_filter)
cbbrain_
len(cbbrain_test.subjects)

In [ ]:
x = torch.zeros(2, 2, 9)
print(x.shape)
key = (slice(0, 0, 0), 0, 2)
x[key]

In [ ]:
labeled_tensor = LabeledTensor(
    dim_names=['subject', 'label', 'stat'],
    dim_keys=[
        [f'subject_{i}' for i in range(10)],
        [f'label_{l}' for l in "ABCD"],
        [stat for stat in ("dice", 'fp', 'tp')]
    ]
)

labeled_tensor.to_dict()

In [ ]:
from abc import ABC, abstractmethod
from inspect import signature
from utils import Config

class Baz(Config):
    def __init__(self, param1: int, param2: str = "asdf"):
        self.param1 = param1
        self.param2 = param2
        
class Bar(Config):
    def __init__(self, param1: str, param2: Baz):
        self.param1 = param1
        self.param2 = param2

class Foo(Config):
    def __init__(self, param1: str, param2: int, param3: Bar):
        self.param1 = param1
        self.param2 = param2
        self.param3 = param3
        

foo = Foo("apple", 10, Bar("orange", Baz(5)))
print(foo.get_config())
print(foo.get_flattened_nested_config())
print(foo)

In [ ]:
combinations([0, 1, 2], [0, 1])

In [ ]:
x = torch.tensor([0, 1, 0, 1])
y = torch.tensor([1, 0, 1, 0])

x[None, :] + y[:, None]

In [ ]:
P, T = 5, 7
prediction = torch.randint(P, size=(1, 5, 5, 5))
target = torch.randint(T, size=(1, 1, 5, 5, 5))

prediction == torch.arange(1, P).reshape(P - 1, 1, 1, 1, 1)
target == torch.arange(1, P).reshape(P - 1, 1, 1, 1, 1)

P_labels = torch.tensor([i // T for i in range(P * T)]).reshape(P * T, 1, 1, 1)
T_labels = torch.tensor([i % P for i in range(P * T)]).reshape(P * T, 1, 1, 1)

torch.cat((prediction, target)).shape

In [ ]:
from itertools import combinations, permutations, product

label_combinations = torch.tensor(list(product(range(P), range(T)))).T
label_combinations[0]

In [ ]:
import xarray as xr
import pandas as pd
import torch

x = xr.DataArray(
    dims=['subject', 'structure', 'stat'],
    coords={
        "subject": [f"subject_{i:02}" for i in range(10)],
        "structure": ['head', 'body', 'tail'],
        "stat": ["dice", "FP", "TP"]
    }
)
x.data = torch.randn(x.data.shape)
x.loc['subject_01', 'head', 'FP'] = 1000

x = x.stack(desired=['structure', 'stat'])

df = x.transpose().to_dataframe(name='what')
df['what']

In [ ]:
import wandb
wandb.init(project="test-project-2")

wandb.log({'table': wandb.Table(dataframe=df['what'])})

In [ ]:
from post_processing import sort_by_size
from skimage.morphology import label, remove_small_holes, dilation


for subject in context.dataset:
    lesion_data = (subject['y'].data[1] > 0.5).numpy()
    img_comp = label(lesion_data)
    _, _, counts = sort_by_size(img_comp)
    
    print(f'subject={subject["name"]}, lesions={len(counts)}, volumes={[v for v in counts]}')
    

In [ ]:
import os
from torch_context import TorchContext
from pathlib import Path
from models import EnsembleOrientations, EnsembleModels

cv_path = "X:\\Checkpoints\\MSSEG2\\cross_validation_01\\ensemble_01\\"

ensemble_path = Path(cv_path)
models = []
for file_path in ensemble_path.iterdir():
    context = TorchContext(
        device, file_path=file_path, variables=dict(DATASET_PATH='X:\\Datasets\\MSSEG2_processed')
    )
    context.keep_components(('model', 'dataset'))
    context.init_components()

    models.append(context.model)

model = EnsembleModels(models, strategy='mean')
model = EnsembleOrientations(model, strategy='mean')
context.model = model
dataset = context.dataset

In [ ]:
from transforms import *
import torchio as tio

subject = dataset.subjects[3]
transforms = tio.Compose([
    SetDataType(torch.float),
    EnforceConsistentAffine(source_image_name='flair_time01'),
    TargetResample(target_spacing=1, tolerance=0.11),
    CropToMask('brain_mask'),
    MinSizePad(96),
    tio.RescaleIntensity((0, 1.), (0.05, 99.5)),
    ConcatenateImages(image_names=["flair_time01", "flair_time02"], image_channels=[1, 1], new_image_name="X"),
    RenameProperty(old_name='ground_truth', new_name='y'),
    CustomOneHot(include="y"),
])

subject = transforms(subject)

t1 = subject['flair_time01'].data
t2 = subject['flair_time02'].data
for img_name, img in subject.get_images_dict().items():
    data = img['data']
    print(img_name, data.min(), data.max(), data.isnan().sum(), data.isinf().sum())
#tio.RescaleIntensity((-1, 1.), (0.05, 99.5))(subject)

In [ ]:
import torchio as tio

for subject in dataset:
    grid_sampler = tio.GridSampler(subject,
                                   patch_size=96,
                                   patch_overlap=32,
                                   padding_mode='edge')
    print(len(grid_sampler))

In [ ]:
import torchio as tio
from ipywidgets import interact
from transforms import *
from models import *
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt

subject = context.dataset.subjects[12]
spatial_augmentations = tio.Compose([
    RandomPermuteDimensions(),
    tio.RandomFlip(axes=(0, 1, 2)),
    tio.OneOf({
        tio.RandomElasticDeformation(): 0.2,
        tio.RandomAffine(scales=0.2, degrees=45, default_pad_value='otsu'): 0.8,
    }, p=0.75),
])

common_transforms_2 = tio.Compose([
    tio.RescaleIntensity((-1, 1.), (0.05, 99.5)),
    ConcatenateImages(image_names=["flair_time01", "flair_time02"], image_channels=[1, 1], new_image_name="X"),
    RenameProperty(old_name='ground_truth', new_name='y'),
    CustomOneHot(include="y"),
])


#augmentations = tio.RandomAffine(scales=0.2, degrees=45)
#augmentations = tio.RandomElasticDeformation()
intensity_augmentations = tio.Compose([
    tio.RandomBiasField(p=0.5), 
    tio.RescaleIntensity((0, 1), (0.01, 99.9)),
    tio.RandomGamma(p=0.8),
    tio.RescaleIntensity((-1, 1)),
    tio.RandomBlur((0, 1), p=0.2),
    tio.RandomNoise(std=0.1, p=0.35)
])

subject = spatial_augmentations(subject)
subject = intensity_augmentations(subject)
subject = common_transforms_2(subject)

img = subject['X'].data[None]
print(img.shape)

vis_features(img)

In [ ]:
def vis_features(x):
    N, C, W, H, D = x.shape
    
    @interact(i=(0, N-1), c=(0, C-1), d=(0, D-1))
    def plot_feature_map(i, c, d):
        fig = plt.figure(figsize=(10, 10))
        plt.imshow(x[i, c, :, :, d].cpu(), cmap="gray")
        plt.colorbar()

In [ ]:
from models import *
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt

def vis_features(x):
    N, C, W, H, D = x.shape
    
    @interact(i=(0, N-1), c=(0, C-1), d=(0, D-1))
    def plot_feature_map(i, c, d):
        fig = plt.figure(figsize=(8, 8))
        plt.imshow(x[i, c, :, :, d].cpu(), cmap="gray")
        plt.colorbar()
        
def vis_model(x, model):
    x = x.to(device)
    modules = list(model.named_modules())
    
    @interact(module=modules[1:])
    def select_module(module):
    
        def forward_module_hook(module, x_in, x_out):
            vis_features(x_out.cpu())
            
        hook_handle = module.register_forward_hook(forward_module_hook)
        with torch.no_grad():
            model(x)
        hook_handle.remove()

model = ModularUNet(
    in_channels=2, 
    out_channels=2, 
    filters=[32, 32, 32, 32, 32, 32], 
    depth=6,
    block_params={
        #'conv_class': WSConv3d,
        #'normalization_class': None,
        'residual': True,
    },
    downsample_class=BlurConv3d,
    downsample_params={'kernel_size': 3, 'stride': 2, 'padding': 1, 'weight_standardization': False},
    upsample_class=BlurConvTranspose3d,
    upsample_params={'kernel_size': 3, 'stride': 2, 'padding': 1, 'output_padding': 0, 'weight_standardization': True},
)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

model = model.to(device)
vis_model(img, model)

In [ ]:
config = load_module("./configs/diffusion_hippocampus.py")

variables = dict(DATASET_PATH="X:/Datasets/Diffusion_MRI/")
context = config.get_context(device, variables)
context.init_components()

In [ ]:
config = load_module("./configs/qsm_deep_grey_matter.py")

variables = dict(DATASET_PATH="X:/Datasets/DGM/segmentation_3T_ps18_v3/")
context = config.get_context(device, variables)
context.init_components()

In [ ]:
config = load_module("./configs/msseg2.py")

variables = dict(DATASET_PATH="X:/Datasets/MSSEG2_processed/")
context = config.get_context(device, variables, fold=2)
context.init_components()

In [ ]:
from data_processing import *
import torchio as tio

config = load_module("./configs/msseg2.py")

variables = dict(DATASET_PATH="X:/Datasets/MSSEG2_processed/")
context = config.get_context(device, variables, fold=2)
context.init_components()

subject_loader = ComposeLoaders([
    ImageLoader(glob_pattern="cv1-ens1-flips-5fold-majority*", image_name='cv1-ens1-flips-5fold-majority', image_constructor=tio.LabelMap,
                label_values={"lesion": 1})
])
context.dataset.load_additional_data('X:/Predictions/MSSEG2/', subject_loader)

context.dataset.all_subjects[0]

In [ ]:
from transforms import *
subjects = [copy.deepcopy(subject) for subject in context.dataset.all_subjects]
for subject in subjects:
    
subjects = [CustomOneHot()(subject) for subject in context.dataset.all_subjects]

In [ ]:
from evaluators import *

seg_eval = SegmentationEvaluator('cv1-ens1-flips-5fold-majority', 'ground_truth')
evaluation = seg_eval(context.dataset.all_subjects)

In [ ]:
import torchio as tio
from transforms import *
from data_processing import *
from evaluators import *
import os

subject_loader = ComposeLoaders([
    ImageLoader(glob_pattern="flair_time01*", image_name='flair_time01', image_constructor=tio.ScalarImage),
    ImageLoader(glob_pattern="flair_time02*", image_name='flair_time02', image_constructor=tio.ScalarImage),
    ImageLoader(glob_pattern="brain_mask.*", image_name='brain_mask', image_constructor=tio.LabelMap,
                label_values={"brain": 1}),
    ImageLoader(glob_pattern="ground_truth.*", image_name="ground_truth", image_constructor=tio.LabelMap,
                label_values={"lesion": 1}
                ),
])
transforms = tio.Compose([
    SetDataType(torch.float),
    EnforceConsistentAffine(source_image_name='flair_time01'),
    TargetResample(target_spacing=1, tolerance=0.11, image_interpolation='bspline'),
    CropToMask('brain_mask'),
])

dataset = SubjectFolder(root="X:/Datasets/MSSEG2_processed/", subject_path="", subject_loader=subject_loader, transforms=transforms)

out_folder = "X:/Datasets/MSSEG2_resampled/"
for subject in dataset:
    subject_folder = os.path.join(out_folder, subject.name)
    if not os.path.exists(subject_folder):
        os.makedirs(subject_folder)
    subject['flair_time01'].save(os.path.join(subject_folder, 'flair_time01_on_middle_space.nii.gz'))
    subject['flair_time02'].save(os.path.join(subject_folder, 'flair_time02_on_middle_space.nii.gz'))
    subject['brain_mask'].save(os.path.join(subject_folder, 'brain_mask.nii.gz'))
    subject['ground_truth'].save(os.path.join(subject_folder, 'ground_truth.nii.gz'))

In [ ]:
lesion_voxels = []
brain_voxels = []
total_voxels = []
for i in range(len(context.dataset)):
    subject = context.dataset[i]
    W, H, D = subject.spatial_shape
    total_voxels.append(W * H * D)
    lesion_voxels.append(subject['y'].data[1].sum().item())
    brain_voxels.append(subject['brain_mask'].data.sum().item())
    if i % 5 == 0:
        print(lesion_voxels, brain_voxels, total_voxels)
    

In [ ]:
1 / (torch.tensor(lesion_voxels) / torch.tensor(brain_voxels)).mean()

In [ ]:
len(context.dataset)

training_dataset = context.dataset.get_cohort_dataset("training")

validation_filter = context.trainer.get_filter_from_scheduled_evaluations(
    context.dataset, 
    context.trainer.validation_evaluators,
)
validation_dataset = context.dataset.get_cohort_dataset(validation_filter)

validation_filter = context.trainer.get_filter_from_scheduled_evaluations(
    context.dataset, 
    context.trainer.validation_evaluators,
    include_fold_filters=False
)

print(validation_filter.filters)
validation_dataset.set_cohort(validation_filter)

from torch.utils.data import DataLoader, SequentialSampler
def dont_collate(subjects):
    return subjects
validation_dataloader = DataLoader(dataset=validation_dataset,
                                   batch_size=16,
                                   sampler=SequentialSampler(validation_dataset),
                                   collate_fn=dont_collate,
                                   num_workers=0)

print(len(training_dataset), len(validation_dataset))

for subjects in validation_dataloader:
    print(subjects)

In [ ]:
import torchio as tio

for i in range(len(context.dataset)):
    
    '''
    untransformed_subject = context.dataset.subjects[i]
    
    image_dict = untransformed_subject.get_images_dict(intensity_only=False)
    
    print(untransformed_subject.name)
    for (image_name1, image1), (image_name2, image2) in combinations(image_dict.items(), 2):
        print(f'{image_name1}.affine - {image_name2}.affine')
        print(image1.affine - image2.affine)'''
    
    subject = context.dataset[i]
    
    print(subject.name)
    print(subject.spacing, subject.spatial_shape)
    
    patch_size=96
    sampler = tio.GridSampler(subject=subject, patch_size=patch_size, patch_overlap=patch_size // 2, padding_mode=None)
    print("num_patches =", len(sampler))

In [ ]:
from transforms import *
import copy

for i in range(len(context.dataset)):
    subject = context.dataset.subjects[i]
    subject = copy.deepcopy(subject)
    
    pre_spacing = subject.get_first_image().spacing
    
    post_spacing = context.dataset[i].get_first_image().spacing
    
    pre_spacing = tuple([round(s, 3) for s in pre_spacing])
    post_spacing = tuple([round(s, 3) for s in post_spacing])
    
    print(pre_spacing, post_spacing)

In [ ]:
import torch
import torchio as tio
from torch.utils.data import DataLoader

patch_size = 96
queue_length = 300
samples_per_volume = 10

sampler = tio.data.UniformSampler(patch_size)
patches_queue = tio.Queue(
    context.dataset,
    queue_length,
    samples_per_volume,
    sampler,
    num_workers=0,
)

patches_loader = DataLoader(dataset=patches_queue, batch_size=2, collate_fn=context.dataset.collate)

num_epochs = 2
model = torch.nn.Identity()
for epoch_index in range(num_epochs):
    for patches_batch in patches_loader:
        print(patches_batch)

In [ ]:
for subject in context.dataset.all_subjects:
    img = subject['flair_time01']
    img.load()
    print(img.spacing)

In [ ]:
import inspect
import os
#print(inspect.getsourcefile(context.model.__class__))

os.getcwd()


In [ ]:
fasd_dataset = context.dataset.get_cohort_dataset('fasd')

In [ ]:
fasd_dataset.preload_and_transform_subjects()

In [ ]:
fasd_dataset

In [ ]:
untransformed_subject = context.dataset.subjects[0]
print("Original labels:")
print(untransformed_subject['whole_roi']['label_values'])

subject = context.dataset[0]
print("\nTransformed labels:")
print(subject['y']['label_values'])

inverse_subject = subject.apply_inverse_transform(warn=False)
print("\nInverse transformed labels:")
print(inverse_subject['y']['label_values'])

In [ ]:
print(subject["X"])

In [ ]:
for key, value in context.dataset[1].items():
    print(key, value, type(value))
    if isinstance(value, dict):
        for key, value2 in value.items():
            print(key, value2)

In [ ]:
def sample_data(loader):
    while True:
        for batch in loader:
            yield batch
           
for i in range(2):
    loader = sample_data(context.dataloader)
    batch = next(loader)
    for key, val in batch.items():
        print(key, val.shape, val.dtype)

In [ ]:
x = torch.randn(20, 20, 20)
for y in x.split([3, 2, 5, 10]):
    print(y.shape)


In [ ]:
import wandb
wandb.run.name

In [ ]:
import wandb
wandb.init(
    project="test-project-2"
    
)

In [ ]:
import random

for i in range(10):
    log_dict = {
        'something': {
            foo: {'mean': random.random(), 'std': random.random()}
            for foo in "ABC"
        }
    }
    wandb.log(log_dict)

In [ ]:
import numpy as np
import pandas as pd

for i in range(10):
    evaluation = {}
    
    for structure in ("A", "B", "C"):
        columns = ["TP", "FP", "TN", "FN", 'dice', "jaccard"]
        S = 25
        subjects = [f'subject_{i:01}' for i in range(S)]

        df = pd.DataFrame(data=np.random.randint(50, 100, size=(S, 4)), columns=["TP", "FP", "TN", "FN"])
        df.insert(loc=0, column='Subject', value=subjects)

        TP, FP, TN, FN = df["TP"], df["FP"], df["TN"], df["FN"]

        df['dice'] = 2 * TP / (2 * TP + FP + FN)
        df['jaccard'] = TP / (TP + FP + FN)
        evaluation[f"Structure {structure}"] = wandb.Table(dataframe=df)
        
    wandb.log({f'Segmentation Evaluation': evaluation})

In [ ]:
wandb.finish()

In [ ]:
from data_processing import *

In [ ]:
%pip install openpyxl

In [ ]:
import pandas as pd

file_path = 'X:\Datasets\Diffusion_MRI\Attributes\demographics.xlsx'
df = pd.read_excel(file_path, index_col=0)
#subject_col = df.columns[0]
data = df.to_dict(orient='dict')

data

In [ ]:
import pathlib
print(pathlib.Path('yourPath.example').suffix) 

In [ ]:
import json

file_path = 'X:\Datasets\DGM\subject_attributes\dgm_label_names.json'
with open(file_path) as f:
    data = json.load(f)
data

In [ ]:
import pandas as pd
file_path = 'X:\Datasets\DGM\subject_attributes\dgm_label_names.csv'
df = pd.read_csv(file_path, index_col=0)
data = df.to_dict()
data

In [ ]:
from utils import is_sequence

def test_func(*args):
    if is_sequence(args) and len(args) == 1 and is_sequence(args[0]):
        args = args[0]
    print(args)
    
test_func("a", "b", "c")
test_func(("a", "b", "c"))
test_func(["a", "b", "c"])



In [ ]:
import torch
a = torch.tensor([1, 0, 1]).bool()
b = torch.tensor([1, 1, 0]).bool()


In [ ]:
import torchio as tio
import torch
from evaluators import SegmentationEvaluator
import pandas as pd

def pd_to_dict(elem):
    if isinstance(elem, dict):
        return {
            key: pd_to_wandb(val)
            for key, val in elem.items()
        }
    elif isinstance(elem, pd.DataFrame):
        return elem.to_dict()
    return elem

def pd_to_wandb(elem):
    if isinstance(elem, dict):
        return {
            key: pd_to_wandb(val)
            for key, val in elem.items()
        }
    elif isinstance(elem, pd.DataFrame):
        return wandb.Table(dataframe=elem)
    return elem

wandb.init(project="test-project-4")
for i in range(10):
    label_values = {letter: val for val, letter in enumerate("ABCDE")}

    subjects = [
        tio.Subject({
            'name': f'subject_{i:02}',
            'pred': tio.LabelMap(
                tensor=torch.randint(0, 5, size=(1, 20, 20, 20)),
                label_values=label_values
            ),
            'target': tio.LabelMap(
                tensor=torch.randint(0, 5, size=(1, 20, 20, 20)),
                label_values=label_values
            ),
        })
        for i in range(20)
    ]

    seg_evaluator = SegmentationEvaluator(
        prediction_label_name='pred', target_label_name='target', stats_to_output=['FP', 'TP', 'dice'], 
    )

    log_dict = seg_evaluator(subjects)
    wandb.log(pd_to_wandb(log_dict))
wandb.finish()

In [ ]:
label_map = tio.LabelMap(
    tensor=torch.randint(0, 5, size=(1, 20, 20, 20)),
    label_values=label_values
)

tio.OneHot()(label_map)

In [ ]:
x = torch.randn(10).unsqueeze(1)
mask = torch.tensor([0, 1, 1, 0]).bool().unsqueeze(0)
x, mask = torch.broadcast_tensors(x, mask)

print(x.shape, mask.shape)

x[mask]

In [ ]:
from evaluators import VolumeStatsEvaluator
import pandas as pd
import torchio as tio


label_values = {f'label_{letter}': (val + 1) for val, letter in enumerate("abcde")}

subjects = [
    tio.Subject({
        'name': f'subject_{i:02}',
        'label': tio.LabelMap(
            tensor=torch.randint(0, 5, size=(1, 20, 20, 20)),
            label_values=label_values
        ),
        'md': tio.ScalarImage(
            tensor=torch.randn(size=(1, 20, 20, 20)) * 2 + 1,
        ),
        'fa': tio.ScalarImage(
            tensor=torch.randn(size=(1, 20, 20, 20)) * 0.5 - 3,
        ),
    })
    for i in range(20)
]

volume_stats_eval = VolumeStatsEvaluator(
    label_map_name='label', image_names=['md', 'fa']
)

volume_stats_eval(subjects)


In [ ]:
from torch.utils.data.dataloader import default_collate

def merge_dicts(a, b):
    merged_dict = {}
    for key in a.keys():
        if key in b:
            merged_dict[key] = merge_dicts(a[key], b[key])
        else:
            merged_dict[key] = a[key]
    for key in b.keys():
        if key not in a:
            merged_dict[key] = b[key]
    return merged_dict

dict1 = {'mean': {'hippo': {'md': {'subject_1': 1}}}}
dict2 = {'mean': {'hippo': {'md': {'subject_2': 2}}}}
dict3 = {'mean': {'hippo': {'fa': {'subject_1': 3}}}}
dict4 = {'mean': {'hippo': {'fa': {'subject_2': 4}}}}

out_dict = merge_dicts(dict1, dict2)

out_dict

In [ ]:
from data_processing import *

train_dataset = context.dataset.get_cohort_dataset(
    RequireAttributes({"pathologies": "None", "protocol": "cbbrain", "rescan_id": "None"})
)
test_dataset = context.dataset.get_cohort_dataset(
    'ab300'
)

print(len(train_dataset), len(test_dataset))



In [ ]:
import numpy as np
for subject in train_dataset.all_subjects:
    img_aff = subject["mean_dwi"].affine
    label_aff = subject['whole_roi'].affine
    diff = img_aff - label_aff
    if np.any(np.abs(diff) > 1e-6):
        print(subject['name'])
        print(diff)

In [ ]:
from utils import save_dataset_as_nn_unet
from data_processing import *

save_dataset_as_nn_unet(
    context.dataset, 
    "X:\\Datasets\\nnUNet_raw_data_base\\nnUNet_raw_data\\Task500_DMRI_Hippocampus_Whole\\",
    short_name="DMRI", image_names=['mean_dwi', 'md', 'fa'], label_map_name='whole_roi',
    train_cohort=RequireAttributes({"pathologies": "None", "protocol": "cbbrain", "rescan_id": "None"}),
    test_cohort=None, #"ab300",
    fix_affine=True
)

In [ ]:
print(context.dataset.all_subjects[0])

In [ ]:
from utils import save_dataset_as_nn_unet

save_dataset_as_nn_unet(
    context.dataset, 
    "X:\\Datasets\\nnUNet_raw_data_base\\nnUNet_raw_data\\Task502_MSSEG2\\",
    short_name="MSSEG2", image_names=['flair_time01', 'flair_time02'], label_map_name='ground_truth',
    train_cohort = 'all'
)

In [ ]:
thing = None
{1:2, **({} if thing is None else thing), 3:4}


In [ ]:
print(RequireAttributes({"pathologies": "None", "protocol": "cbbrain", "rescan_id": "None"}))

In [ ]:
import copy
import torchio as tio

subject = context.dataset.all_subjects_map['cbbrain_192']
subject = copy.deepcopy(subject)

img_aff = subject["mean_dwi"].affine
label_aff = subject['whole_roi'].affine
print(img_aff - label_aff)

subject['whole_roi'].affine = subject['mean_dwi'].affine

img_aff = subject["mean_dwi"].affine
label_aff = subject['whole_roi'].affine
print(img_aff - label_aff)

subject['whole_roi'].save('test_label.nii.gz')

loaded_label = tio.LabelMap('test_label.nii.gz')
print(img_aff - loaded_label.affine)
